In [2]:
pip install torch torchvision torchaudio transformers scikit-learn pandas numpy matplotlib seaborn streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.

In [14]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

# Load Dataset
df = pd.read_csv("/content/TASK-ML-INTERN.csv")  # Replace with actual dataset path

# Separate Features and Target
X = df.drop(columns=['hsi_id', 'vomitoxin_ppb']).values  # Spectral reflectance values
y = df['vomitoxin_ppb'].values  # Target variable

# Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Reshape Input to (samples, sequence_length=1, embed_dim=448)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])  # Shape: (num_samples, 1, num_wavelengths)

# Convert to PyTorch Tensors
X_tensor = torch.tensor(X_reshaped, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split into Train & Test Sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Define Dataset Class
class SpectralDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create Dataloaders
BATCH_SIZE = 32
train_dataset = SpectralDataset(X_train, y_train)
test_dataset = SpectralDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [15]:
import torch.nn as nn
import torch.optim as optim

class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super(TransformerRegressor, self).__init__()

        # Transformer Encoder Layer
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,  # Must match embed_dim
            nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        # Fully Connected Layer
        self.fc = nn.Linear(input_dim, 1)  # Regression Output

    def forward(self, x):
        x = self.transformer_encoder(x)  # Transformer Encoder
        x = torch.mean(x, dim=1)  # Pooling over sequence
        x = self.fc(x)  # Regression Output
        return x.squeeze()

# ✅ Fix input_dim (must match embed_dim)
device = "cuda" if torch.cuda.is_available() else "cpu"
input_dim = X_reshaped.shape[2]  # 448 spectral bands
model = TransformerRegressor(input_dim=input_dim).to(device)


In [16]:
# Define Loss Function & Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Train Model
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")


Epoch 1, Loss: 148855949.6923
Epoch 2, Loss: 147862083.3654
Epoch 3, Loss: 150152554.6538
Epoch 4, Loss: 148533479.0000
Epoch 5, Loss: 147752794.0385
Epoch 6, Loss: 166217335.5962
Epoch 7, Loss: 148402364.0385
Epoch 8, Loss: 148129645.0385
Epoch 9, Loss: 147468825.6538
Epoch 10, Loss: 151664653.6442
Epoch 11, Loss: 147452564.2308
Epoch 12, Loss: 165690487.7692
Epoch 13, Loss: 147424896.6538
Epoch 14, Loss: 147353641.8077
Epoch 15, Loss: 148032830.6923
Epoch 16, Loss: 150991323.8654
Epoch 17, Loss: 165540249.3077
Epoch 18, Loss: 147739994.1250
Epoch 19, Loss: 165433597.1346
Epoch 20, Loss: 146828402.1827


In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, test_loader, device):
    """Evaluates the trained Transformer model on test data."""
    model.eval()  # Set model to evaluation mode
    y_pred_list, y_actual_list = [], []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            preds = model(batch_X)  # Get predictions

            y_pred_list.extend(preds.cpu().numpy())  # Move predictions to CPU
            y_actual_list.extend(batch_y.cpu().numpy())  # Move actual values to CPU

    # Compute Metrics
    mae = mean_absolute_error(y_actual_list, y_pred_list)
# Compute RMSE manually instead of using 'squared=False'
    rmse = mean_squared_error(y_actual_list, y_pred_list) ** 0.5  # Square root to get RMSE

    r2 = r2_score(y_actual_list, y_pred_list)

    print(f"📊 **Transformer Model Evaluation:**")
    print(f"✅ MAE:  {mae:.4f}")
    print(f"✅ RMSE: {rmse:.4f}")
    print(f"✅ R² Score: {r2:.4f}")

    return mae, rmse, r2

# Run Evaluation
mae, rmse, r2 = evaluate_model(model, test_loader, device)


📊 **Transformer Model Evaluation:**
✅ MAE:  4297.3800
✅ RMSE: 17244.5947
✅ R² Score: -0.0638
